<span style="font-size:150%">라이브러리</span>

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision import datasets
from torch.utils.data import DataLoader

<span style="font-size:150%">경로 및 파라미터</span>

In [2]:
data_path = './data/cifar10'
save_path = './parameters/'
epochs = 10
batch_size = 64
learning_rate = 0.001

device = 'cuda' if torch.cuda.is_available() else 'cpu'

<span style="font-size:150%">어그멘테이션</span>

In [3]:
train_transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

test_transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)

<span style="font-size:150%">데이터셋</span>

In [4]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

train_data = datasets.CIFAR10(
    root=data_path,
    train=True,
    download=False,
    transform=train_transform
)
test_data = datasets.CIFAR10(
    root=data_path,
    train=False,
    download=False,
    transform=test_transform
)

<span style="font-size:150%">데이터로더</span>

In [5]:
trainloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
testloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

<span style="font-size:150%">모델 설계</span>

In [6]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))

        self.fc = nn.Linear(32*8*8, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        flatten = out.view(out.size(0), -1)
        score = self.fc(flatten)
        return score

<span style="font-size:150%">모델 및 손실함수, 옵티마이저 선언</span>

In [7]:
mode = 1

if mode==1:
    model = ConvNet()
elif mode==2:
    model = models.resnet18(weights=None)
    model.fc = nn.Linear(512, 10)
elif mode==3:
    model = models.resnet18(weights=None)
    model_weight = torch.load(os.path.join(save_path, 'resnet18.pth'))
    model.load_state_dict(model_weight)
    model.fc = nn.Linear(512, 10)

model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

<span style="font-size:150%">모델 학습 함수</span>

In [8]:
def train(epoch):
    print('\nEpoch: %d'%epoch)
    # model train mode로 전환
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    total = 0
    for (inputs, labels) in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        _, pred = torch.max(outputs, 1)
        total += outputs.size(0)
        running_acc += (pred == labels).sum().item()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    total_loss = running_loss / len(trainloader)
    total_acc = 100 * running_acc / total
    print(f'Train epoch : {epoch} loss : {total_loss} Acc : {total_acc}%')

<span style="font-size:150%">모델 평가 함수</span>

In [9]:
def test(epoch):
    print('\nEpoch: %d'%epoch)
    # model eval mode로 전환
    model.eval()
    running_loss = 0.0
    running_acc = 0.0
    total = 0
    with torch.no_grad():
        for (inputs, labels) in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, pred = torch.max(outputs, 1)
            total += outputs.size(0)
            running_acc += (pred == labels).sum().item()
            loss = criterion(outputs, labels)
            running_loss += loss.item()
        total_loss = running_loss / len(testloader)
        total_acc = 100 * running_acc / total
        print(f'Test epoch : {epoch} loss : {total_loss} Acc : {total_acc}%')

<span style="font-size:150%">모델 학습 및 평가</span>

In [10]:
for epoch in range(epochs):
    train(epoch)
    test(epoch)
    path = os.path.join(save_path, f'epoch_{epoch}.pth')
    torch.save(model.state_dict(), path)


Epoch: 0
Train epoch : 0 loss : 2.2586585594260176 Acc : 18.808%

Epoch: 0
Test epoch : 0 loss : 2.1949669661795257 Acc : 24.46%

Epoch: 1
Train epoch : 1 loss : 2.117654431201613 Acc : 25.512%

Epoch: 1
Test epoch : 1 loss : 2.041436091350142 Acc : 27.71%

Epoch: 2
Train epoch : 2 loss : 2.006399275091908 Acc : 28.996%

Epoch: 2
Test epoch : 2 loss : 1.9603045761205589 Acc : 30.39%

Epoch: 3
Train epoch : 3 loss : 1.9398119914562195 Acc : 31.48%

Epoch: 3
Test epoch : 3 loss : 1.9129172905235534 Acc : 32.66%

Epoch: 4
Train epoch : 4 loss : 1.8975174274590925 Acc : 33.134%

Epoch: 4
Test epoch : 4 loss : 1.8783943136786199 Acc : 33.82%

Epoch: 5
Train epoch : 5 loss : 1.8647721580532202 Acc : 34.27%

Epoch: 5
Test epoch : 5 loss : 1.8509801018769574 Acc : 34.52%

Epoch: 6
Train epoch : 6 loss : 1.8389792937756804 Acc : 35.374%

Epoch: 6
Test epoch : 6 loss : 1.8202099891225243 Acc : 35.74%

Epoch: 7
Train epoch : 7 loss : 1.8114417890453582 Acc : 36.386%

Epoch: 7
Test epoch : 7 loss